In [1]:
import pandas as pd
import numpy as np
import itertools
import os
import networkx as nx
import re
from pprint import pprint
from copy import deepcopy
from itertools import product
from time import time
%matplotlib inline
from collections import OrderedDict, defaultdict

In [139]:
df = pd.read_csv("order_brush_order.csv")

In [140]:
df.event_time = pd.to_datetime(df.event_time)

In [141]:
df = df.sort_values(by='event_time')

In [145]:
df.groupby(by=['event_time', 'shopid']).orderid.count()[df.groupby(by=['event_time', 'shopid']).orderid.count()>1]

event_time           shopid   
2019-12-27 00:01:13  147941492    2
2019-12-27 00:02:36  26974701     2
2019-12-27 00:02:39  147941492    2
2019-12-27 00:02:47  147941492    2
2019-12-27 00:03:22  147941492    2
                                 ..
2019-12-31 22:31:57  147941492    2
2019-12-31 22:47:05  147941492    2
2019-12-31 23:02:43  7190564      2
2019-12-31 23:04:44  61556313     2
2019-12-31 23:44:49  143281052    2
Name: orderid, Length: 420, dtype: int64

In [146]:
df = df.set_index('event_time')

In [6]:
df.orderid.nunique()

222750

In [171]:
my_unique = lambda x: len(np.unique(x))
concentrate = lambda sample: sample.orderid.rolling('1H', closed='both').count()/sample.userid.rolling('1H', closed='both').apply(my_unique)
 
def get_order_brushing_user(shopid):
    sample = df.loc[df.shopid==shopid].copy(deep=True)
    sample = sample.groupby(by='event_time').agg({"orderid":"count","userid":"count"})
    c_rate = concentrate(sample)
    if sum(concentrate(sample)>=3)>0:        
        sample['c_rate'] = list(concentrate(sample))
#         prop = sample.groupby(by=['userid']).rolling('1H').orderid.count().div(sample.reset_index().groupby(by="event_time").orderid.count().rolling('1H').sum(), level='event_time').reset_index()
        brush_index = sample.loc[sample.c_rate>=3].index    
        sample = pd.concat([sample.loc[(i-pd.Timedelta('1H')):i] for i in brush_index]).sort_index()
#         prop = sample.groupby(by=['userid']).rolling('1H', closed='both').orderid.count().reset_index().groupby(by='userid').orderid.sum()/sample.rolling('1H', closed='both').orderid.count().sum()
        prop= sample.groupby(by=['userid']).orderid.count()
        return list(np.sort(list(prop[prop==prop.max()].index)))
#         return list(prop[prop.event_time.isin(sample[sample.c_rate>=3].index) & (prop.orderid==prop.orderid.max())].userid)
    else:
        return '0'

In [180]:
sample = df[df.shopid==147941492].reset_index()

In [177]:
sample = sample.groupby(by='event_time').agg({"orderid":"count","userid":"count"})

In [181]:
sample.groupby(by='event_time').orderid.count()

event_time
2019-12-27 00:00:08    1
2019-12-27 00:00:12    1
2019-12-27 00:00:31    1
2019-12-27 00:00:32    1
2019-12-27 00:00:33    1
                      ..
2019-12-31 23:58:17    1
2019-12-31 23:58:28    1
2019-12-31 23:58:59    1
2019-12-31 23:59:42    1
2019-12-31 23:59:47    1
Name: orderid, Length: 11388, dtype: int64

In [178]:
concentrate(sample)

event_time
2019-12-27 00:00:08     1.0
2019-12-27 00:00:12     2.0
2019-12-27 00:00:31     3.0
2019-12-27 00:00:32     4.0
2019-12-27 00:00:33     5.0
                       ... 
2019-12-31 23:58:17    91.0
2019-12-31 23:58:28    92.0
2019-12-31 23:58:59    92.0
2019-12-31 23:59:42    93.0
2019-12-31 23:59:47    94.0
Length: 11388, dtype: float64

In [166]:
sample['c_rate'] = list(concentrate(sample))

ValueError: window must be an integer

In [163]:
sample = sample.groupby(by='event_time').agg({"orderid":"count","userid":"count"})

In [155]:
sample.head(50)

,event_time,orderid,shopid,userid
0,2019-12-27 00:00:08,31075208826809,147941492,195903090
1,2019-12-27 00:00:12,31075212032449,147941492,210365192
2,2019-12-27 00:00:31,31075231334015,147941492,306545
3,2019-12-27 00:00:32,31075232254299,147941492,202519981
4,2019-12-27 00:00:33,31075232409868,147941492,200500884
5,2019-12-27 00:00:37,31075236524199,147941492,88862740
6,2019-12-27 00:00:40,31075240500903,147941492,86692729
7,2019-12-27 00:00:43,31075243964273,147941492,197551102
8,2019-12-27 00:00:47,31075247660252,147941492,178608922
9,2019-12-27 00:00:51,31075250193276,147941492,208222979


In [149]:
sample.orderid.rolling('1H', closed='both').count()

event_time
2019-12-27 00:00:08     1.0
2019-12-27 00:00:12     2.0
2019-12-27 00:00:31     3.0
2019-12-27 00:00:32     4.0
2019-12-27 00:00:33     5.0
                       ... 
2019-12-31 23:58:17    91.0
2019-12-31 23:58:28    92.0
2019-12-31 23:58:59    92.0
2019-12-31 23:59:42    93.0
2019-12-31 23:59:47    94.0
Name: orderid, Length: 11703, dtype: float64

In [56]:
c_rate = concentrate(sample)

In [57]:
sample['c_rate'] = list(concentrate(sample))

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [58]:
brush_index = sample.loc[sample.c_rate>=3].index

In [60]:
sample = pd.concat([sample.loc[(i-pd.Timedelta('1H')):i] for i in brush_index]).sort_index()

In [61]:
sample

,orderid,shopid,userid,c_rate
event_time,,,,
2019-12-27 00:20:33,31076433523765,181009364,214208720,1.0
2019-12-27 00:30:03,31077003494175,181009364,214208720,2.0
2019-12-27 00:53:26,31078405144719,181009364,214208720,3.0
2019-12-29 13:54:31,31298070356347,181009364,101832161,1.0
2019-12-29 13:54:52,31298092385907,181009364,101832161,2.0
2019-12-29 13:55:51,31298150262970,181009364,101832161,3.0


In [68]:
prop= sample.groupby(by=['userid']).orderid.count()#.reset_index().groupby(by='userid').orderid.sum()

In [69]:
list(np.sort(list(prop[prop==prop.max()].index)))

[101832161, 214208720]

In [172]:
%time brushing = pd.Series(df.shopid.unique()).apply(get_order_brushing_user)

KeyboardInterrupt: 

In [127]:
[i for i in brushing[brushing!=0] if len(i)>1]

[[5307816, 214808165],
 [101832161, 214208720],
 [1974334, 107414154],
 [29857724, 212200633],
 [35639374, 159315857],
 [187697407, 215009429]]

In [173]:
brushing[brushing!=0] = list(['&'.join([str(j) for j in i]) for i in brushing[brushing!=0]])

In [174]:
prev = pd.read_csv("31GB_raven4.csv").set_index('shopid')

In [175]:
pd.DataFrame({"shopid":df.shopid.unique(), "userid":brushing}).set_index("shopid").to_csv("31GB_raven4.csv")

In [96]:
pdf.userid[pdf.userid!=0]

shopid
68862371      67554410
27476241     174145893
192608876    213646699
10402            77819
114919380    117199298
               ...    
203070501    189834273
156995951    204919399
129113839    215408773
123401085     12869645
155016230    188431260
Name: userid, Length: 264, dtype: object

In [103]:
prev.userid = prev.userid.replace('0',0)

In [106]:
sum([prev.userid.iloc[i] == pdf.userid.iloc[i] for i in range(len(pdf))])

18770

In [201]:
sample = df[df.shopid==181009364]

In [202]:
c_rate = concentrate(sample)

In [203]:
sample['c_rate'] = list(c_rate)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [211]:
sample.index

DatetimeIndex(['2019-12-27 00:20:33', '2019-12-27 00:30:03',
               '2019-12-27 00:53:26', '2019-12-28 01:33:18',
               '2019-12-28 10:51:30', '2019-12-28 22:49:57',
               '2019-12-29 00:05:30', '2019-12-29 13:54:31',
               '2019-12-29 13:54:52', '2019-12-29 13:55:51',
               '2019-12-30 06:58:51', '2019-12-30 16:05:56',
               '2019-12-30 20:43:43', '2019-12-31 12:31:38',
               '2019-12-31 13:50:08', '2019-12-31 20:05:50'],
              dtype='datetime64[ns]', name='event_time', freq=None)

In [210]:
sample.index[2] = pd.to_datetime('2019-12-27 01:20:33') 

TypeError: Index does not support mutable operations

In [204]:
sample.il

,orderid,shopid,userid,c_rate
event_time,,,,
2019-12-27 00:20:33,31076433523765,181009364,214208720,1.0
2019-12-27 00:30:03,31077003494175,181009364,214208720,2.0
2019-12-27 00:53:26,31078405144719,181009364,214208720,3.0
2019-12-28 01:33:18,31167197128282,181009364,27847950,1.0
2019-12-28 10:51:30,31200690834058,181009364,214515524,1.0
2019-12-28 22:49:57,31243796889716,181009364,160784036,1.0
2019-12-29 00:05:30,31248330354574,181009364,18028327,1.0
2019-12-29 13:54:31,31298070356347,181009364,101832161,1.0
2019-12-29 13:54:52,31298092385907,181009364,101832161,2.0


In [205]:
sample[sample.c_rate>=3]

,orderid,shopid,userid,c_rate
event_time,,,,
2019-12-27 00:53:26,31078405144719,181009364,214208720,3.0
2019-12-29 13:55:51,31298150262970,181009364,101832161,3.0


In [207]:
sample

,orderid,shopid,userid,c_rate
event_time,,,,
2019-12-27 00:20:33,31076433523765,181009364,214208720,1.0
2019-12-27 00:30:03,31077003494175,181009364,214208720,2.0
2019-12-27 00:53:26,31078405144719,181009364,214208720,3.0
2019-12-28 01:33:18,31167197128282,181009364,27847950,1.0
2019-12-28 10:51:30,31200690834058,181009364,214515524,1.0
2019-12-28 22:49:57,31243796889716,181009364,160784036,1.0
2019-12-29 00:05:30,31248330354574,181009364,18028327,1.0
2019-12-29 13:54:31,31298070356347,181009364,101832161,1.0
2019-12-29 13:54:52,31298092385907,181009364,101832161,2.0


In [110]:
x = ['2019-12-31 20:05:50', '2019-12-31 21:05:49', '2019-12-31 21:05:50']

In [112]:
y = pd.DataFrame({'y':[1,1,1]}, index=x)

In [114]:
y.index = pd.to_datetime(y.index)

In [115]:
y

,y
2019-12-31 20:05:50,1
2019-12-31 21:05:49,1
2019-12-31 21:05:50,1


In [117]:
y.sort_index().rolling(window='1H', ).sum()

,y
2019-12-31 20:05:50,1.0
2019-12-31 21:05:49,2.0
2019-12-31 21:05:50,2.0
